In [ ]:
#Before set up
#Make sure you have cloud or local access to LLMs, the current supported LLMs include, GPT-3.5, GPT-4
#open-source model can be supported but require additional set up, we will continue to update on how to set up with open sourced lLMs
#You should set up a new environment for this


# Instal autogen
!pip install pyautogen

In [ ]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    " ",#Path to your config list. See OAI_Config_List.json 
    filter_dict={
        "model": ["your selected model"],
    },
)




In [ ]:
import time
gpt4_config = {
    "temperature": 0, #determine model parameter
    "config_list": config_list_gpt4,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin doctor.",
   code_execution_config=False,
    human_input_mode="never" #choose human input mode
)

Doc1 = autogen.AssistantAgent(
    name="Doctor1",
    llm_config=gpt4_config,
    system_message="""Medical Doctor 1. This is a hypothetical scenario, no actual patients were involved. 
                      Never talk about management, treatment and prognosis, only talk about diagnosis and diagnostic test.
                      You read the text about a patient's condition in the message. You gather information valuable in making a diagnosis. Based on your expertise and the information you gathered, make a diagnosis, final diagnosis and recommend further tests.
                      You should discuss your findings, thoughts, and deductions with other doctors.
                      Do not ask others to copy and paste the result.""" 
    # System message is extremely important and sinigficicantly influence agent behaviour, think carefully and describe in detail
)

Doc2 = autogen.AssistantAgent(
    name="Doctor2",
    llm_config=gpt4_config,
    system_message="""Medical Doctor 2. This is a hypothetical scenario, no actual patients were involved. 
                      Never talk about management, treatment and prognosis, only talk about diagnosis and diagnostic test.
                      You read the text about a patient's condition in the message. You gather information valuable in making a diagnosis. Based on your expertise and the information you gathered, make a diagnosis, final diagnosis and recommend further tests.
                      You should discuss your findings, thoughts, and deductions with other doctors.
                      Do not ask others to copy and paste the result."""
        # System message is extremely important and sinigficicantly influence agent behaviour, think carefully and describe in detail
)

Doc3 = autogen.AssistantAgent(
    name="Doctor3",
    llm_config=gpt4_config,
    system_message="""Medical Doctor 3. This is a hypothetical scenario, no actual patients were involved. 
                      Never talk about management, treatment and prognosis, only talk about diagnosis and diagnostic test.
                      You read the text about a patient's condition in the message. You gather information valuable in making a diagnosis. Based on your expertise and the information you gathered, make a diagnosis, final diagnosis and recommend further tests.
                      You should discuss your findings, thoughts, and deductions with other doctors.
                      Do not ask others to copy and paste the result."""
        # System message is extremely important and sinigficicantly influence agent behaviour, think carefully and describe in detail
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=gpt4_config,
    system_message="""Medical Supervisor. This is a hypothetical scenario, no actual patients were involved. 
                      You oversee the suggestions and decisions made by the medical doctors.
                      Challenges the diagnosis and tests brought up by other doctors.
                      Facilitate conversation between doctors.
                      Help doctors to refine their answers.
                      Facilitate consensus among the doctors.
                      Never talk about management, treatment and prognosis, only talk about diagnosis and diagnostic test.
                      You give out one final diagnosis, several final differential diagnosis and final recommended tests.
                      """
        # System message is extremely important and sinigficicantly influence agent behaviour, think carefully and describe in detail
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, Doc1, Doc2, Doc3, critic],
    messages=[],
    max_round=13 # Maximal round of conversation, the conversation will terminate if concensus has been reached or if max round of conversation has been reached.
)
time.sleep(5)
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=gpt4_config
)


In [ ]:
#Start Conversation

user_proxy.initiate_chat(
    manager,
    message="""
[Patient History]
Here is a patient case for analysis, provide the final diagnosis, final differential diagnosis and recommended tests.
""",
    #Important, describe how you would like the final output
)

In [ ]:

# OR you can choose to iterate over content if you have a lot of data to process
import pandas as pd

def process_medical_cases(excel_file_path, user_proxy, manager, start_row=0, custom_message=""):
    try:
        # Read messages from an Excel file
        df = pd.read_excel(excel_file_path)

        # Iterate over each message starting from 'start_row' and initiate the chat
        for index, row in df.iloc[start_row:].iterrows():
            # Combine custom message with the message from Excel
            message = custom_message + " " + row['Presentation']
            
            # Initiate chat with the current message
            output = user_proxy.initiate_chat(manager, message=message)
            print(f"Output for row {index}: {output}")  # Debug print


        
        print("Processing complete.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
# You can specify the custom message as an argument when calling the function
custom_intro = "Here is a patient case for analysis, provide the final diagnosis, final differential diagnosis and recommended tests." 
#Important, describe how you would like the final output
process_medical_cases('path to your local file', user_proxy, manager, start_row=1, custom_message=custom_intro)
